In [1]:
# Generative classifiers in LDA, QDA and Naive Bayes

In [28]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split

wine = datasets.load_wine()
X, y = wine.data, wine.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [35]:
a = np.zeros((3 , 2 , 2))
a

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [36]:
class LDA:
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.N, self.D = self.X.shape
        self.classes, self.counts = np.unique(y, return_counts = True)

        # priors
        self.priors = self.counts/self.N
        ## Get mu for each class and overall Sigma
        self.mu_ks = []
        self.Sigma = np.zeros((self.D, self.D))        
        for i, k in enumerate(self.classes):
            
            X_k = self.X[self.y == k]
            mu_k = X_k.mean(0).reshape(self.D, 1)
            self.mu_ks.append(mu_k)

            for x_n in X_k:
                x_n = x_n.reshape(-1,1)
                x_n_minus_mu_k = (x_n - mu_k)
                self.Sigma += np.dot(x_n_minus_mu_k, x_n_minus_mu_k.T)
            
        self.Sigma /= self.N
        
    def _mvn_density(self, x_n, mu_k, Sigma):
        x_n_minus_mu_k = (x_n - mu_k)
        density = np.exp(-(1/2)*x_n_minus_mu_k.T @ np.linalg.inv(Sigma) @ x_n_minus_mu_k)
        return density
        


        
    def classify(self, X_test):
        # X_test is a matrix 
        y_n = []
        for i, x_n in enumerate(X_test):
            x_n = x_n.reshape(-1,1)
            pks = np.zeros(len(self.classes))

            for j, k in enumerate(self.classes):
                px_given_y = self._mvn_density(x_n, self.mu_ks[j], self.Sigma)
                py_prior = self.priors[j]
                py_given_x = px_given_y * py_prior
                pks[j] = py_given_x
            y_n.append(self.classes[np.argmax(pks)])
        return y_n
            

        

In [72]:
lda = LDA()
lda.fit(X, y)
yhat = lda.classify(X)
np.mean(yhat == y)


1.0

In [47]:
class QDA:
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.N, self.D = self.X.shape
        self.classes, self.counts = np.unique(self.y, return_counts = True)

        # priors
        self.priors = self.counts/self.N

        # muks and sigmaks
        self.mu_ks = []
        self.Sigma_ks = np.zeros((len(self.classes), self.D, self.D))
        for i, k in enumerate(self.classes):
            X_k = self.X[self.y == k]
            mu_k = X_k.mean(0).reshape(self.D, 1)
            self.mu_ks.append(mu_k)
            sigma_k = np.zeros((self.D, self.D))

            # loop over each data in class k
            for j, x_n in enumerate(X_k):
                # calculate sigma k
                x_n = x_n.reshape(-1, 1)
                xn_minus_muk = x_n - mu_k
                sigma_k += xn_minus_muk @ xn_minus_muk.T
            sigma_k /= self.counts[i]
            self.Sigma_ks[i] = sigma_k
    def _mvn_density(self, x_n, Sigma_k, mu_k):
        xn_minus_muk = x_n - mu_k
        density = np.linalg.det(Sigma_k) ** (-.5) * np.exp( -.5 * (xn_minus_muk.T @ np.linalg.inv(Sigma_k) @ xn_minus_muk) )
        return density
    def classify(self, X_test):
        y_n = []
        for i, x_n in enumerate(X_test):
            x_n = x_n.reshape(-1,1)
            pks = np.zeros(len(self.classes))

            for j, k in enumerate(self.classes):
                px_given_y = self._mvn_density(x_n, self.Sigma_ks[j], self.mu_ks[j])
                prior_k = self.priors[j]
                py_given_x = prior_k * px_given_y
                pks[j] = py_given_x
            y_n.append(self.classes[np.argmax(pks)])
        return y_n

In [71]:
qda = QDA()
qda.fit(X, y)
yhat = qda.classify(X)
np.mean(yhat == y)

0.9943820224719101

In [64]:
class NaiveBayes:
    def _estimate_parameters(self, X_k):
        class_parameters = []
        # each 
        for d in range(self.D):
            x_kd = X_k[:,d]
            if self.distributions[d] == 'normal':
                mu_d = x_kd.mean()
                std2 = np.var(x_kd)
                class_parameters.append([mu_d, std2])
            elif self.distributions[d] == 'bernoulli':
                # pk
                p = x_kd.mean()
                class_parameters.append(p)
            elif self.distributions[d] == 'poisson':
                lam = np.mean(x_kd)
                class_parameters.append(lam) 
        return class_parameters

   
    def fit(self, X, y, distributions = None):
        self.X, self.y = X, y
        self.N, self.D = self.X.shape
        self.classes, self.counts = np.unique(self.y, return_counts = True)
        if distributions is None:
            distributions = ['normal' for i in range(len(y))] 
            # may be wrong to use y instead of self.D in range(len(y)), 
            # to be checked out 
        self.distributions = distributions
        self.priors = self.counts/self.N

        self.parameters = []
        for i, k in enumerate(self.classes):
            X_k = self.X[y == k]
            self.parameters.append(self._estimate_parameters(X_k)) 
            
        
        
    def _get_probabilities(self, x_n, cls):
        class_parameters = self.parameters[cls]
        class_probability = 1
        for i in range(self.D):
            x_nd = x_n[i]
            if self.distributions[i] == 'normal':
                mu, var = class_parameters[i]
                class_probability *= (var)**(-.5) * np.exp( -(x_nd - mu)**2/var )
            elif self.distributions[i] == 'bernoulli':
                p = class_parameters[i]
                class_probability *= (p**x_nd)*(1-p)**(1-x_nd)
            elif self.distributions[i] == 'poisson':
                lam = class_parameters[i]
                class_probability *= np.exp(-lam)*lam**x_nd
        return class_probability
        
    def classify(self, X_test):
        
        y_n = []
        for i, x_n in enumerate(X_test):
            x_n = x_n.reshape(-1,1)
            pks = np.zeros(len(self.classes))

            for j, k in enumerate(self.classes):
                px_given_y = self._get_probabilities(x_n, k)
                prior_k = self.priors[j]
                py_given_x = prior_k * px_given_y
                pks[j] = py_given_x
            y_n.append(self.classes[np.argmax(pks)])
        return y_n

In [65]:
nb = NaiveBayes()
nb.fit(X, y)
yhat = nb.classify(X)
np.mean(yhat == y)

0.9775280898876404

**Implementation** 

In [66]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

lda = LinearDiscriminantAnalysis()
lda.fit(X, y);

qda = QuadraticDiscriminantAnalysis()
qda.fit(X, y);

nb = GaussianNB()
nb.fit(X, y);

In [68]:
np.mean(nb.predict(X) == y)

0.9887640449438202

In [69]:
np.mean(qda.predict(X) == y)


0.9943820224719101

In [70]:
np.mean(lda.predict(X) == y)

1.0